In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import os

In [ ]:
class ASLSignRecognizer:
    def __init__(self, img_size=(64, 64), num_classes=29):
        self.img_size = img_size
        self.num_classes = num_classes
        self.model = self._build_model()

    def _build_model(self):
        model = models.Sequential([
            layers.Input(shape=(self.img_size[0], self.img_size[1], 3)),
            layers.Conv2D(32, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(self.num_classes, activation='softmax')
        ])

        model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])
        return model

    def preprocess_image(self, image_path):
        img = cv2.imread(image_path)
        img = cv2.resize(img, self.img_size)
        img = img.astype('float32') / 255.0
        return img

    def train(self, X_train, y_train, epochs=10, batch_size=32):
        # Convert labels to numpy array if they're not already
        y_train = np.array(y_train)

        # Create data augmentation
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True
        )

        # Train the model
        self.model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                      epochs=epochs,
                      validation_split=0.2)

    def predict(self, image_path):
        img = self.preprocess_image(image_path)
        img = np.expand_dims(img, axis=0)
        prediction = self.model.predict(img)
        return np.argmax(prediction)

    def save_model(self, filepath):
        self.model.save(filepath)

    def load_model(self, filepath):
        self.model = tf.keras.models.load_model(filepath)

In [ ]:
def class_to_label(class_id):
    if class_id < 26:  # Letters A-Z
        return chr(65 + class_id)  # Converts 0-25 to 'A'-'Z'
    elif class_id == 26:  # Space
        return " "
    elif class_id == 27:  # Delete
        return "DEL"
    else:  # Nothing
        return "NOTHING"

In [ ]:
import pandas as pd
dataset=pd.read_csv('asl_dataset.csv')

# Initialize recognizer
recognizer = ASLSignRecognizer()

# Train the model
recognizer.train(X_train, y_train, epochs=10)

# Save the model
recognizer.save_model('asl_model.h5')

# Load the model
recognizer.load_model('asl_model.h5')

# Predict on a new image
prediction = recognizer.predict("asl_image.jpg")
print(f"Predicted sign: {class_to_label(prediction)}")